In [ ]:
import numpy as np, torch, time, os
from Ctubes.geometry_utils import compute_plane_normals, regular_polygon
from Ctubes.plot_utils import plot_generatrix
from Ctubes.tubes import Directrix, Generatrix, CTube
from Ctubes.target_cross_sections import fix_end_cross_sections
from Ctubes.opt import CTubeOptimizationProblem
from Ctubes.misc_utils import load_curve_from_obj
from Ctubes.path_utils import get_name, setup_paths

paths = setup_paths(get_name())

PI = np.pi
TORCH_DTYPE = torch.float64
torch.set_default_dtype(TORCH_DTYPE)
torch.set_printoptions(precision=4)

%load_ext autoreload
%autoreload 2

%matplotlib widget

# Figure 10: Non-Planar Generatrices

## #1: Planar Generatrix (Reference)

In [ ]:
# Define a path to output specific to the current test case
paths = setup_paths(get_name(), test_name="fig10_nonplanar_planar")

In [ ]:
# Directrix: read curve from file
cps_ref = load_curve_from_obj(os.path.join(paths["data"], "fig10_nonplanar/directrix_Q.obj"))
K = cps_ref.shape[0]
M = K

In [ ]:
# Generatrix: regular N-gon
N = 4
tube_radius = 2.0
generatrix_2d = regular_polygon(N, tube_radius)

fig, ax = plot_generatrix(generatrix_2d)

In [ ]:
# C-tube
directrix = Directrix(cps_ref, M)

X0 = directrix.X[0]
T0 = directrix.get_tangents()[0]
generatrix = Generatrix(generatrix_2d, X0, T0)

tube = CTube(directrix, generatrix)

In [ ]:
tube.has_planar_profile()

In [ ]:
fig, ax = tube.plot_3d()

In [ ]:
fig, ax = tube.plot_unrolled_strips()

### Optimization

In [ ]:
directrix.aabb_diagonal_length() ** 2

In [ ]:
# Set up optimization problem

opt_weights = {
    'match_target_cross_sections': 1e2 / generatrix.aabb_diagonal_length() ** 2,
}

objective_args = {
    'target_cross_sections': fix_end_cross_sections(tube),
}

opt_prob = CTubeOptimizationProblem(
    tube, 
    opt_weights, 
    objective_args,
)

opt_prob.activate_cps(False)
opt_prob.activate_theta(False)
opt_prob.activate_apex_loc_func(False)
opt_prob.activate_plane_normals(True)

In [ ]:
opt_prob.opt_weights['smooth_plane_normal_diffs'] = 0.0  # deactivate plane normal smoothing to avoid confounding the comparison between planar and nonplanar

In [ ]:
opt_prob.compute_objective(print_to_console=True)

In [ ]:
# Save initial state
paths_init = setup_paths(get_name(), test_name="fig10_nonplanar_init")

opt_prob.save_meshes(paths=paths_init)
opt_prob.save_optimization_results(paths=paths_init)

In [ ]:
from scipy.optimize import minimize
from Ctubes.opt import obj_and_grad

torch.autograd.set_detect_anomaly(False)

# Set up optimization configuration
opt_prob.configure_optimization_output(paths)

# Get initial parameters
params0 = opt_prob.get_params_numpy()

# Define objective and gradient function for SciPy
obj_and_grad_scipy = lambda params: obj_and_grad(params, opt_prob)

# Fix variables via double-sided bounds
fixed_indices = []  # no fixed DOF by default

bounds = [(None, None)] * len(params0)
for idx in fixed_indices:
    bounds[idx] = (params0[idx], params0[idx])
print(f"Fixing {len(fixed_indices)} parameters.")

# Add initial state to history
opt_prob.add_objective_to_history()

In [ ]:
# Run optimization
start_time = time.time()
result = minimize(
    obj_and_grad_scipy, 
    params0, 
    jac=True, 
    method='L-BFGS-B',
    options={'ftol': 1.0e-10, 'gtol': 1.0e-5, 'disp': True, 'maxiter': 2000},
    bounds=bounds,
    callback=opt_prob.optimization_callback,
)
result.execution_time = time.time() - start_time

# Finalize optimization (save results, render videos, cleanup)
opt_prob.finalize_optimization(result)

In [ ]:
opt_prob.compute_objective(print_to_console=True)

In [ ]:
fig, ax = opt_prob.plot_objective_history()

In [ ]:
fig, ax = opt_prob.plot_3d()

In [ ]:
fig, ax = opt_prob.plot_unrolled_strips()

In [ ]:
# Find non-planar cross-sections
from Ctubes.geometry_utils import point_cloud_is_planar

ctube_vertices = opt_prob.tube_network.compute_vertices()[0]
non_planar_cross_sections = []
for i in range(M):
    pts = ctube_vertices[i]
    if not point_cloud_is_planar(pts):
        non_planar_cross_sections.append(i)

# Find points at which the plane normals differ
plane_normals = opt_prob.tube_network.tubes[0].get_plane_normals()
distinct_plane_normals = []
if not opt_prob.tube_network.has_planar_profile():
    for i in range(M):
        all_same = True
        for j in range(1, N):
            if not torch.allclose(plane_normals[i][j], plane_normals[i][0]):
                all_same = False
                break
        if not all_same:
            distinct_plane_normals.append(i)

print(f"Non-planar cross-sections: \n{non_planar_cross_sections}")
print(f"Distinct plane normals: \n{distinct_plane_normals}")

## #2: Non-Planar Generatrix

In [ ]:
# Define a path to output specific to the current test case
paths = setup_paths(get_name(), test_name="fig10_nonplanar_nonplanar")

In [ ]:
# Generatrix: regular N-gon
N = 4
tube_radius = 2.0
generatrix_2d = regular_polygon(N, tube_radius)

fig, ax = plot_generatrix(generatrix_2d)

In [ ]:
# Directrix: read curve from file
cps_ref = load_curve_from_obj(os.path.join(paths["data"], "fig10_nonplanar/directrix_Q.obj"))
K = cps_ref.shape[0]
M = K

In [ ]:
# C-tube
directrix = Directrix(cps_ref, M)

X0 = directrix.X[0]
T0 = directrix.get_tangents()[0]
generatrix = Generatrix(generatrix_2d, X0, T0)

plane_normals_ref = compute_plane_normals(directrix.X, kind='bisecting', closed_curve=directrix.closed_curve)
plane_normals_per_ridge = []
for i in range(N):
    pn = plane_normals_ref.clone()
    plane_normals_per_ridge.append(pn)
plane_normals_ref = torch.swapaxes(torch.stack(plane_normals_per_ridge), 0, 1)  # shape (M, N, 3), one plane per generatrix point

tube = CTube(directrix, generatrix, plane_normals_ref)

In [ ]:
tube.has_planar_profile()

Plot

In [ ]:
fig, ax = tube.plot_3d()

In [ ]:
fig, ax = tube.plot_unrolled_strips()

### Optimization

In [ ]:
# Set up optimization problem

opt_weights = {
    'match_target_cross_sections': 1e2 / generatrix.aabb_diagonal_length() ** 2,
}

objective_args = {
    'target_cross_sections': fix_end_cross_sections(tube),
}

opt_prob = CTubeOptimizationProblem(
    tube, 
    opt_weights, 
    objective_args,
)

opt_prob.activate_cps(False)
opt_prob.activate_theta(False)
opt_prob.activate_apex_loc_func(False)
opt_prob.activate_plane_normals(True)

In [ ]:
opt_prob.opt_weights['smooth_plane_normal_diffs'] = 0.0  # deactivate plane normal smoothing to avoid confounding the comparison between planar and nonplanar

In [ ]:
opt_prob.compute_objective(print_to_console=True)

In [ ]:
from scipy.optimize import minimize
from Ctubes.opt import obj_and_grad

torch.autograd.set_detect_anomaly(False)

# Set up optimization configuration
opt_prob.configure_optimization_output(paths)

# Get initial parameters
params0 = opt_prob.get_params_numpy()

# Define objective and gradient function for SciPy
obj_and_grad_scipy = lambda params: obj_and_grad(params, opt_prob)

# Fix variables via double-sided bounds
fixed_indices = []  # no fixed DOF by default

bounds = [(None, None)] * len(params0)
for idx in fixed_indices:
    bounds[idx] = (params0[idx], params0[idx])
print(f"Fixing {len(fixed_indices)} parameters.")

# Add initial state to history
opt_prob.add_objective_to_history()

In [ ]:
# Run optimization
start_time = time.time()
result = minimize(
    obj_and_grad_scipy, 
    params0, 
    jac=True, 
    method='L-BFGS-B',
    options={'ftol': 1.0e-10, 'gtol': 1.0e-5, 'disp': True, 'maxiter': 2000},
    bounds=bounds,
    callback=opt_prob.optimization_callback,
)
result.execution_time = time.time() - start_time

# Finalize optimization (save results, render videos, cleanup)
opt_prob.finalize_optimization(result)

In [ ]:
opt_prob.compute_objective(print_to_console=True)

In [ ]:
fig, ax = opt_prob.plot_objective_history()

In [ ]:
fig, ax = opt_prob.plot_3d()

In [ ]:
fig, ax = opt_prob.plot_unrolled_strips()

In [ ]:
# Find non-planar cross-sections
from Ctubes.geometry_utils import point_cloud_is_planar

ctube_vertices = opt_prob.tube_network.compute_vertices()[0]
non_planar_cross_sections = []
for i in range(M):
    pts = ctube_vertices[i]
    if not point_cloud_is_planar(pts):
        non_planar_cross_sections.append(i)

# Find points at which the plane normals differ
plane_normals = opt_prob.tube_network.tubes[0].get_plane_normals()
distinct_plane_normals = []
if not opt_prob.tube_network.has_planar_profile():
    for i in range(M):
        all_same = True
        for j in range(1, N):
            if not torch.allclose(plane_normals[i][j], plane_normals[i][0]):
                all_same = False
                break
        if not all_same:
            distinct_plane_normals.append(i)

print(f"Non-planar cross-sections: \n{non_planar_cross_sections}")
print(f"Distinct plane normals: \n{distinct_plane_normals}")